In [1]:
import pandas as pd
import re
import random

In [3]:
df = pd.read_csv('../Data/quadruplet_test_pred.csv')
df.head(1)

,original_id,content,clean_tweet,final_sentiment,labels,quadruplet,spam,sentiment_label,pred_quadruplet_pt_bart,pred_quadruplet_pt_t5,pred_quadruplet_tf_t5
0,1.640000e+18,@tokopedia min aku udah bayar tapi kenapa diba...,min aku udah bayar tapi kenapa dibatalin pesan...,negative,payment; produk;,"(pesananku, udah bayar tapi kenapa dibatalin, ...",NaN,sentiment,"(_, kenapa dibatalin pesanan ku, neutral, pay...","(pesananku, udah bayar tapi kenapa dibatalin, ...","(pesanan, dibatalin pesanan, negative, delivery);"


In [4]:
df.shape

(39, 11)

In [5]:
i = random.randint(0, len(df))
row = df.iloc[i]
text = row['clean_tweet']
ori_quadruplet = row['quadruplet']
pred_quadruplet = row['pred_quadruplet_pt_t5']
print(text)
print(ori_quadruplet)
print(pred_quadruplet)

Pake tokped karena ongkirnya murah sekarang malah jadi mehong kek shopee
(tokped, ongkirnya sekarang malah mehong, negative, delivery);
(tokped, ongkirnya sekarang malah mehong, negative, delivery);


In [6]:
def extract_quadruplet(sequence):
    extractions = []
    # find all matching quadruplet with (); pattern
    quadruplets = re.findall("\(.*?\)", sequence)
    for quadruplet in quadruplets:
        # Remove the in the start "("  and at the end ")".
        quadruplet = quadruplet[1:-1]
        try:
            aspect_term, opinion_term, sentiment, aspect_category = quadruplet.split(', ')
        except ValueError:
            aspect_term, opinion_term, sentiment, aspect_category = '', '', '', ''
        aspect_term = aspect_term.strip().lower()
        opinion_term = opinion_term.strip().lower()
        sentiment = sentiment.strip().lower()
        aspect_category = aspect_category.strip().lower()
        extractions.append((aspect_term, opinion_term, sentiment, aspect_category)) 
    return extractions

In [7]:
i = random.randint(0, len(df))
quad = df.iloc[i]['pred_quadruplet_pt_t5']
print(quad)
extraction = extract_quadruplet(quad)
print(extraction)

(akun shopee, gak bisa transaksi, negative, website&apps);
[('akun shopee', 'gak bisa transaksi', 'negative', 'website&apps')]


In [8]:
# == Metrics ==
precision_fn = lambda n_tp, n_pred: float(n_tp) / float(n_pred) if n_pred != 0 else 0
recall_fn = lambda n_tp, n_gold: float(n_tp) / float(n_gold) if n_gold != 0 else 0
f1_fn = (
    lambda precision, recall: (2 * precision * recall) / (precision + recall)
    if precision != 0 or recall != 0
    else 0
)


def score(pred, gold):
    assert len(pred) == len(gold)
    n_tp, n_gold, n_pred = 0, 0, 0

    for i in range(len(pred)):
        n_gold += len(gold[i])
        n_pred += len(pred[i])

        for t in pred[i]:
            if t in gold[i]:
                n_tp += 1
            

    precision = precision_fn(n_tp, n_pred)
    recall = recall_fn(n_tp, n_gold)
    f1 = f1_fn(precision, recall)

    return {"precision": precision, "recall": recall, "f1": f1}

In [9]:
# == Evaluation ==
#def evaluate(pred_seqs, gold_seqs, postprocessor: IPostprocess, sents, implicit=False):
def evaluate(pred_seqs, gold_seqs):
    assert len(pred_seqs) == len(gold_seqs)
    num_samples = len(gold_seqs)

    all_labels, all_preds = [], []

    for i in range(num_samples):
        gold_list = extract_quadruplet(gold_seqs[i])
        pred_list = extract_quadruplet(pred_seqs[i])

        all_labels.append(gold_list)
        all_preds.append(pred_list)

    raw_scores = score(all_preds, all_labels)
    return raw_scores, all_labels, all_preds
    #all_fixed_preds = postprocessor.check_and_fix_preds(all_preds, sents, implicit)
    #fixed_scores = score(all_fixed_preds, all_labels)

    #return raw_scores, fixed_scores, all_labels, all_preds, all_fixed_preds

In [10]:
raw_scores, all_labels, all_preds = evaluate(df['pred_quadruplet_pt_t5'], df['quadruplet'])

In [11]:
raw_scores

{'precision': 0.9411764705882353, 'recall': 0.96, 'f1': 0.9504950495049505}

In [53]:
i = random.randint(0, len(df))
print(all_preds[i])
print(all_labels[i])

[('login tokped', 'bisa dibantu gaa', 'neutral', 'website&apps')]
[('login tokped', 'bisa dibantu gaa', 'neutral', 'website&apps')]
